# Wrangle Exercises

### Data Acquisition
- These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.
- This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

1. Read the case, department, and source data into their own spark dataframes.

2.  Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

- Inspect your folder structure. What do you notice?

3. Inspect the data in your dataframes. 
- Are the data types appropriate? 
- Write the code necessary to cast the values to the appropriate types.

-----------

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

2. How many Stray Animal cases are there?

3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

4. Convert the council_district column to a string column.

5. Extract the year from the case_closed_date column.

6. Convert num_days_late from days to hours in new columns num_hours_late.

7. Join the case data with the source and department data.

8. Are there any cases that do not have a request source?

9. What are the top 10 service request types in terms of number of requests?

10. What are the top 10 service request types in terms of average days late?

11. Does number of days late depend on department?

12. How do number of days late depend on department and request type?


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [2]:
# Read in CSV file 
source = (spark.read.csv("source.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [3]:
# Another way to read in data:

# (
#     spark.read.format("csv")
#     .option("sep", ",")
#     .option("inferSchema", True)
#     .option("header", True)
#     .load("source.csv")
# )

In [4]:
# from pyspark.sql.types import StructType, StructField, StringType

# schema = StructType(
#     [
#         StructField("source_id", StringType()),
#         StructField("source_username", StringType()),
#     ]
# )


# # Read csv, but now we specify the schema:

# source = spark.read.csv("source.csv", header=True, schema=schema)

In [5]:
# Read in CSV file 
case = (spark.read.csv("case.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [6]:
# Read in CSV file 
dept = (spark.read.csv("dept.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [7]:
source.write.json('source_json', mode='overwrite')
source.write.json('case_json', mode='overwrite')
source.write.json('dept_json', mode='overwrite')

In [8]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [9]:
# shape of dataframe
len(source.columns), source.count() 

(2, 140)

In [10]:
# shape of dataframe
len(case.columns), case.count() 

(14, 841704)

In [11]:
# shape of dataframe
len(dept.columns), dept.count() 

(4, 39)

In [14]:
source.dtypes, dept.dtypes, case.dtypes

([('source_id', 'string'), ('source_username', 'string')],
 [('dept_division', 'string'),
  ('dept_name', 'string'),
  ('standardized_dept_name', 'string'),
  ('dept_subject_to_SLA', 'string')],
 [('case_id', 'int'),
  ('case_opened_date', 'string'),
  ('case_closed_date', 'string'),
  ('SLA_due_date', 'string'),
  ('case_late', 'string'),
  ('num_days_late', 'double'),
  ('case_closed', 'string'),
  ('dept_division', 'string'),
  ('service_request_type', 'string'),
  ('SLA_days', 'double'),
  ('case_status', 'string'),
  ('source_id', 'string'),
  ('request_address', 'string'),
  ('council_district', 'int')])

In [18]:
case.show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [ ]:
# Rename 'SLA_due_date' to 'case_due_date' using .withColumnRenamed

df = df.withColumnRenamed('SLA_due_date', 'case_due_date')

In [ ]:
# use .withColumn to change columns from string to boolean values

df = df.withColumn('case_closed', expr('case_closed == "YES"'))\
.withColumn('case_late', expr('case_late == "YES"'))

In [ ]:
# council_district as a string instead of int
df = df.withColumn('council_district', col('council_district').cast('string'))

In [ ]:
# to_timestamp, fmt

fmt = "M/d/yy H:mm"

df = df.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))\
.withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))\
.withColumn('case_due_date', to_timestamp('case_due_date', fmt))

In [ ]:
# request_address: trim and lowercase

df = df.withColumn('request_address', lower(trim('request_address')))

In [ ]:
# convert the number of days a case is late to a number of weeks

df = df.withColumn('num_weeks_late', expr('num_days_late / 7'))

In [ ]:
# use format_string function to pad zeros for council_district
df = df.withColumn('council_district', format_string('%03d', col('council_district').cast('int')))

In [ ]:
# create a new column for zipcode:

df = df.withColumn('zipcode', expr('right(request_address, 5)'))

In [ ]:
# join the df and dept dataframe using 'dept_division' as common key
# drop columns as needed (keep standardized_dept_name)
# convert dept_subject_to_SLA to boolean

df = (
    df
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

In [ ]:
df.show(1, False, True)